In [2]:
import numpy as np
import pandas as pd 
import os 
import cv2 
import pickle

In [3]:
data = pickle.load(open('./Pillow_01/img/Results.pickle', mode = 'rb'))
data

{'data': [array([[ 0.0860999 ,  0.17186214, -0.1215091 ,  0.02226814, -0.02793001,
           0.24380986,  0.04825497, -0.04956462, -0.13407592,  0.10700487,
           0.06258755, -0.02195659,  0.13064966, -0.15982366, -0.05222454,
          -0.04774049, -0.02668801,  0.15378569, -0.03221204,  0.05691868,
           0.0605196 , -0.11773016,  0.02434952,  0.00818192,  0.01072925,
          -0.11938998, -0.12703618, -0.12288352, -0.02125017,  0.1196753 ,
           0.01084544,  0.07194949, -0.08492116,  0.09404994,  0.04972649,
          -0.05114644, -0.00975055, -0.05010111,  0.0938315 , -0.08576045,
          -0.05593767, -0.21726611,  0.06212515,  0.05796819, -0.09010557,
          -0.1466589 ,  0.10661316,  0.09551126, -0.1624385 ,  0.14330885,
           0.13394822, -0.01613589, -0.05402068,  0.02452856, -0.06449176,
           0.00145379, -0.00178286, -0.01828158,  0.04255519, -0.09493184,
          -0.03127246,  0.05273642,  0.13054629, -0.22612804,  0.03948915,
          -0.0333

In [4]:
x = np.array(data['data'])
y = np.array(data['label'])
x.shape

(10, 1, 128)

In [5]:
x = x.reshape(-1,128)
x.shape

(10, 128)

In [6]:
y.shape

(10,)

In [7]:
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test =  train_test_split(x,y, train_size = 0.8, test_size = 0.2, random_state = 0)
x_train.shape

(8, 128)

In [8]:
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [9]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train)

LogisticRegression()

In [10]:
def get_report(model, x_train, y_train, x_test, y_test):
    y_pre_train  = model.predict(x_train)
    y_pre_test = model.predict(x_test)
    #accuracy 
    acc_train = accuracy_score(y_train,y_pre_train)
    acc_test = accuracy_score(y_test,y_pre_test)

    #f1 score
    f1_score_train = f1_score(y_train,y_pre_train, average="macro" )
    f1_score_test = f1_score(y_test,y_pre_test, average="macro")
    # print
    print('Accurency Train = %0.2f'%acc_train)
    print('Accurency Test = %0.2f'%acc_test)
    print('F1 Score Train = %0.2f'%f1_score_train)
    print('F1 Score Test = %0.2f'%f1_score_test)

In [11]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.88
Accurency Test = 0.50
F1 Score Train = 0.76
F1 Score Test = 0.33


In [12]:

# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [13]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 1.00
F1 Score Train = 1.00
F1 Score Test = 1.00


In [14]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [15]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 1.00
F1 Score Train = 1.00
F1 Score Test = 1.00


In [16]:

# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [17]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.75
Accurency Test = 0.00
F1 Score Train = 0.53
F1 Score Test = 0.00


In [ ]:
#Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=2, n_jobs=1, verbose=2)

In [19]:
model_grid.fit(x_train, y_train)

Fitting 2 folds for each of 216 candidates, totalling 432 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[

C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=7, svm__gam

C:\Users\ASUS\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=2,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [20]:
model_best_estimator = model_grid.best_estimator_
model_best_estimator

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(C=3, gamma=0.1, probability=True)),
                             ('rf',
                              RandomForestClassifier(max_depth=3,
                                                     n_estimators=5))],
                 voting='soft', weights=[2, 3, 1])

In [ ]:
pickle.dump(model_best_estimator, open("./Pillow_01/img/ml_face_person_identity.pkl", mode='wb'))
     